# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [3]:
import pandas as pd
import numpy as np
from scipy import stats

In [4]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [5]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [6]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

### Question 1 - What test is appropriate for this problem? Does CLT apply?

- The appropriate test for this problem is a comparison of proportion, where we measure the total call backs for white and black sounding names and divide by the total amount of resumes with black and white sounding names.
- The CLT applies in this case as the amount of success and failure are larger than 10 and we can consider independent as the researchers performed a randomized experiments. Therefore, we can assume that our data is normally distributed.

### Question 2 - What are the null and alternate hypotheses?

##### Hypotheses
- Null Hypotheses - P1 = P2 - Black sounding name receives the same amount of calls back as white sounding name
- Alternative Hypotheses - P1 ≠ P2 - Black sounding name receives different amount of callbacks than white sounding name

### Question 3 - Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

##### Frequentist Approach

In [28]:
#Black sounding name

total_b = len(b.race)
b_call = np.sum(b.call)
b_prop = np.mean(b.call)

print('n = {}'.format(total_b))
print('Callback = {}'.format(b_call))
print('P1 = {}'.format(b_prop))

n = 2435
Callback = 157.0
P1 = 0.0644763857126236


In [29]:
#White sounding name

total_w = len(w.race)
w_call = np.sum(w.call)
w_prop = np.mean(w.call)

print('n = {}'.format(total_w))
print('Callback = {}'.format(w_call))
print('P2 = {}'.format(w_prop))

n = 2435
Callback = 235.0
P2 = 0.09650924056768417


In [63]:
prop = np.mean(data.call)
# (b_prop - w_prop) / np.sqrt(((prop*(1-prop)) * ((1/total_b)+(1/total_w))))

In [60]:
# Difference of proportion --> P1 - P2
diff_prop = b_prop - w_prop

# Difference Variance --> Var(P1) + Var(P2)
var_b = (b_prop*(1-b_prop))/(total_b)
var_w = (w_prop*(1-w_prop))/(total_w)

diff_var = np.sqrt(var_b + var_w)

z_value = diff_prop/diff_var
p_value = stats.norm.cdf(z_value)


print('z-value | {}'.format(z_value))
print('p-value | {}'.format(p_value))

z-value | -4.115550519002299
p-value | 1.931281906454845e-05


In [66]:
# Confidence Interval & Margin of Error

prop_moe =  1.96 * diff_var # 1.96 is the z-value for a 95% confidence interval 
prop_ci_pos = diff_prop +  prop_moe
prop_ci_neg = diff_prop -  prop_moe

print('Confidence Interval | {},{}'.format(prop_ci_neg, prop_ci_pos))
print('Margin of Error | ±{}'.format(prop_moe))

Confidence Interval | -0.0472882612037449,-0.016777448506376254
Margin of Error | ±0.015255406348684322


##### Bootstrap Approach (Two Sample Hypotheses)

In [67]:
# Bootstrap Function

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates with for loop
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data))) 

    return bs_replicates


# Bootstrap Replicates
b_replicates = draw_bs_reps(b.call, np.mean, 1000)
w_replicates = draw_bs_reps(w.call, np.mean, 1000)

z_test = stats.ttest_ind(b_replicates,w_replicates)
print('z-value | {}'.format(z_test[0]))
print('p-value | {}'.format(z_test[1]))

z-value | -128.85063687655423
p-value | 0.0


In [70]:
# Confidence Interval & Margin of Error

percentiles = [2.5, 97.5]

diff_means = b_replicates - w_replicates
mean_reps = np.mean(diff_means)

bs_ci = np.percentile(diff_means, percentiles)
bs_moe = np.std(diff_means) * 1.96 # 1.96 is the z-value for a 95% confidence level.

print("Confidence Interval | {}".format(bs_ci))
print("Margin of Error | {}".format(bs_moe))

Confidence Interval | [-0.04722792 -0.01724846]
Margin of Error | 0.015133391948656823


### Question 4 - Write a story describing the statistical significance in the context or the original problem.

There is no doubt that racial discrimination still occurs and affects the lives of people in their daily life, but is your name/race affecting you when you are sending out resumes? Our recent findings show that yes, your name has a significant effect on whether you are going to land that job interview or not.

When randomly assigning identical resumes with black-sounding names and white sounding names to request interview, white-sounding names received callbacks 9.65% of the time while black sounding name received callbacks only 6.48% of the time. In addition, our findings show that this discrepancy did not happen by mere chance. 

Our statistics show that with a p-value of 1.931281906454845e-05 we can reject the hypothesis that a black sounding name will receive the same chances of a white sounding name with over 99% confidence. 

### Question 5 - Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis cannot conclude that race/name is the single and most important for a callback success. Our analysis shows that race/name plays an important role whether you will receive a callback for an interview, but other factors such as education, age, experiences were not accounted in (identical resumes were sent). Therefore, to conclude that race/name is the most important factor for a callback success, more analysis comparing the difference of other variables need to be done.